In [1]:
from datasets import load_dataset

imdb = load_dataset("imdb")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset imdb downloaded and prepared to /home/upbeat_northcutt/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 75.94it/s]


In [2]:
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

2023-04-06 07:58:57.627516: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 07:58:58.693009: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-06 07:58:58.693161: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-06 07:58:58.693173: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [4]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [10]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/upbeat_northcutt/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-52408368931d496e.arrow
Loading cached processed dataset at /home/upbeat_northcutt/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-6283df0c106d702d.arrow


In [11]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [13]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert_results_imdb/checkpoint-4500", num_labels=2, id2label=id2label, label2id=label2id
).to('cuda')

In [17]:
training_args = TrainingArguments(
    output_dir="bert_results_imdb",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    warmup_ratio = 0.06,
    max_steps = 5000,
    optim = 'adamw_torch',
    save_strategy = 'steps',
    evaluation_strategy='steps',
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train('bert_results_imdb/checkpoint-4500')

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
max_steps is given, it will override any value given in num_train_epochs
Loading model from bert_results_imdb/checkpoint-4500.
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 25000
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5000
 

Step,Training Loss,Validation Loss,Accuracy
5000,0.121700,0.256555,0.928560


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 16
Saving model checkpoint to bert_results_imdb/checkpoint-5000
Configuration saved in bert_results_imdb/checkpoint-5000/config.json
Model weights saved in bert_results_imdb/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in bert_results_imdb/checkpoint-5000/tokenizer_config.json
Special tokens file saved in bert_results_imdb/checkpoint-5000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from bert_results_imdb/checkpoint-1500 (score: 0.19886040687561035).


TrainOutput(global_step=5000, training_loss=0.012169338989257813, metrics={'train_runtime': 282.4683, 'train_samples_per_second': 283.218, 'train_steps_per_second': 17.701, 'total_flos': 1.0488689973064128e+16, 'train_loss': 0.012169338989257813, 'epoch': 3.2})

In [63]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert_results_imdb/checkpoint-4000", num_labels=2, id2label=id2label, label2id=label2id
)

loading configuration file bert_results_imdb/checkpoint-4000/config.json
Model config DistilBertConfig {
  "_name_or_path": "bert_results_imdb/checkpoint-4000",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading weights file bert_results_imdb/checkpoint-4000/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequence

In [64]:
model.push_to_hub('romainf/distilbert-base-uncased-imdb-4000')

Configuration saved in /tmp/tmpasg1p7ws/config.json
Model weights saved in /tmp/tmpasg1p7ws/pytorch_model.bin
Uploading the following files to romainf/distilbert-base-uncased-imdb-4000: config.json,pytorch_model.bin


CommitInfo(commit_url='https://huggingface.co/romainf/distilbert-base-uncased-imdb-4000/commit/0382d936d0cdd7d0e2dcd43b6d3198bd01e1b5fa', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='0382d936d0cdd7d0e2dcd43b6d3198bd01e1b5fa', pr_url=None, pr_revision=None, pr_num=None)